In [ ]:
correct_word = "kapitulashon"
wrong_word = "kapiutalshon"

In [ ]:
from re import sub


class GST:
    def __init__(self, tag=None):
        self.children = {}
        self.tag = []

        # Careful with value of zero being interpreted as false
        if tag is not None:
            # print(f"Initializing new node with tag {tag}")
            self.tag.append(tag)
        # else:
            # print(f"Initializing new node without tag {tag}")
        
    def __str__(self):
        if self.children:
            ret = f"({len(self.children)}"
            for k, v in self.children.items():
                ret += f" {k}:{v.tag} {v}"
            return ret + ")"
        else:
            return ""

    def insert(self, word, tag):
        current = self
        word_length = len(word)
        letters_found = 0

        # contiue until the whole word was matched
        while letters_found < word_length:
            found = False
            # check for unmatched part of word of decreasing size until something is found
            for j in range(word_length - letters_found):
                k = word_length - j
                sub_word = word[letters_found: k]

                if sub_word in current.children:
                    # print(f"{sub_word} in {str(current)}")
                    current = current.children[sub_word]
                    letters_found = k
                    found = True
                    # if this is the whole word, add a tag to the current node
                    if tag not in current.tag and letters_found == word_length:
                        current.tag.append(tag)

                if not found:
                    # print(f"{sub_word} not in {str(current)}")
                    sub_word_length = len(sub_word)
                    found_key = ""
                    rest_part = ""
                    for key, node in current.children.items():
                        key_length = len(key)
                        if sub_word_length < key_length and sub_word == key[:sub_word_length]:
                            rest_part = key[sub_word_length:]
                            found_key = key
                            found = True
                            break
                    # if the word was a subpart of a child node
                    if found:
                        # split the node into two and append the tag if the final letters are less than the found node
                        # in which case you are done
                        if word_length - letters_found < len(found_key):
                            new_node = GST(tag)
                        else:
                            new_node = GST()
                            rest_of_word = word[letters_found + sub_word_length:]
                            new_node.children[rest_of_word] = GST(tag=tag)
                        new_node.children[rest_part] = current.children[found_key]
                        del current.children[found_key]
                        current.children[sub_word] = new_node
                        return
            # insert a new node if nothing was found
            if not found:
                rest_word = word[letters_found:]
                current.children[rest_word] = GST(tag=tag)
                # print(f"setting child {rest_word}:{tag} when letters found is: {word[:letters_found]}")
                # print("s", current.children[rest_word].tag, self )
                break


    def findLongestSubstring(self, index1, index2):
        current = self
        longest = ""
        found_in_child = False
        for k, child in current.children.items():
            found, child_longest = child.findLongestSubstring(index1, index2)
            full_substring = k + child_longest
            if len(full_substring) > len(longest):
                if found or (index1 in child.tag and index2 in child.tag):
                    found_in_child = True
                    longest = full_substring

        if not current.children:
            if index1 in current.tag and index2 in current.tag:
                return True, ""
            else:
                return False, ""
        return found_in_child, longest
        
    def findMatch(self, word):            
        word_length = len(word)
        found_match_indices = []
        print(f"Finding matches for {word}")

        search_suffix = word
        print(f"Looking for suffix {search_suffix}")
        
        current = self
        search_suffix_length = len(search_suffix)
        letters_found = 0

        # contiue until the whole word was matched
        while letters_found < search_suffix_length:
            found = False
            # check for unmatched part of word of decreasing size until something is found
            for m in range(search_suffix_length - letters_found):
                k = search_suffix_length - m
                sub_word = word[letters_found: k]

                if sub_word in current.children:
                    print(f"{sub_word} in {str(current)}")
                    current = current.children[sub_word]
                    letters_found = k
                    found = True
                    # if this is the whole word, add a tag to the current node
                    if letters_found == search_suffix_length:
                        found_match_indices += map(lambda x: (x, search_suffix_length), current.tag)
                        print(f"Found all letters: {found_match_indices}")
                        
                        break
            
                # if not found:
                    # print(f"{sub_word} not in {str(current)}")
                    # sub_word_length = len(sub_word)
                    # found_key = ""
                    # rest_part = ""
                    # for key, node in current.children.items():
                    #     key_length = len(key)
                    #     if sub_word_length < key_length and sub_word == key[:sub_word_length]:
                    #         rest_part = key[sub_word_length:]
                    #         found_key = key
                    #         found = True
                    #         break
                    # # if the word was a subpart of a child node
                    # if found:
                    #     # split the node into two and append the tag if the final letters are less than the found node
                    #     # in which case you are done
                    #     # if search_suffix_length - letters_found < len(found_key):
                    #     found_match_indices += map(lambda x: (x, letters_found + sub_word_length), current.children[found_key].tag)
                    #     # else:
                    #     #     new_node = GST()
                    #     break
            if not found:
                # rest_word = word[letters_found:]
                # current.children[rest_word] = GST(tag=tag)
                # print(f"setting child {rest_word}:{tag} when letters found is: {word[:letters_found]}")
                # print("s", current.children[rest_word].tag, self )
                break
        return found_match_indices


    def populate(self, words, case=False):
        for i, word in enumerate(words):
            
            if not case:
                word = word.lower()

            word_length = len(word)
            for j in range(word_length):
                k = word_length - j - 1
                # print(f"inserting {word[k:]}")
                self.insert(word[k:], i)
                # print(f"new tree {self}")

In [ ]:
import pandas as pd
import sys, os
sys.path.append(os.path.abspath('../code'))

# hny_pap_nl = pd.read_csv("../data/hny/pap-nl.csv", na_filter=False)
nbo_pap = pd.read_csv("../data/nbo/pap(cap).csv", na_filter=False)
corpus = nbo_pap
words = corpus["pap-simple"].values



In [ ]:
t = GST()
# words = ["a", "aba", "ashon"]
# words = ["ab", "a"]#, "ashon"]
# t.populate(words)
# t.populate(["bananaa", "moana"])
# t.populate(["ananaa", "ana"])
# t.populate(["ananaa"])
# t.populate(["anaa", "mana"])
# t.populate(["aa", "aab"])
# t.populate(["anaa"])
# t.populate(["ana", "a"])
# t.populate(words)
# t.insert("aaa", 0)
# t.insert("aa", 1)
# t.insert("ab", 0)
# t.insert("aba", 1)

t.insert("black", 0)
t.insert("lack", 0)
t.insert("lock", 1)

print(t)
print(t.findLongestSubstring(0, 1))
print(t.findMatch("black"))
print(t.findMatch("lack"))
print(t.findMatch("ock"))